In [ ]:
import os
import cv2
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import os
import dlib
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
mask_model = load_model("mask_detector.model")

shape_predictor_path="shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(shape_predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=224)

In [2]:
prototxtPath = os.path.sep.join(['face_detector', "deploy.prototxt"])
weightsPath = os.path.sep.join(['face_detector',"res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
def face_detect_and_aligned(image):
    # show the original input image and detect faces in the grayscale
    # image
    img=image
#     img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
    (300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    face_detection = net.forward()
    bbox=[]
    faces=[]
    prediction=[]
    #to draw faces on image
    for i in range(face_detection.shape[2]):
        confidence = face_detection[0, 0, i, 2]
        if confidence > 0.8:
            box = face_detection[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            (X,Y,X1,Y1) = (max(0, x), max(0, y),min(w-1, x1), min(h-1, y1))
            rect = dlib.rectangle(x,y,x1,y1)
            faceAligned = fa.align(img, gray, rect)
            cv2.imwrite('facealigned'+str(i)+'.jpg',faceAligned)
            face = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2RGB)
            cv2.imwrite('face_resized'+str(i)+'.jpg',face)
#             face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face) 
            bbox.append((x,y,x1,y1))
        if len(faces)>0:
            faces = np.array(faces, dtype="float32")
            prediction = mask_model.predict(faces, batch_size=32)
        return (bbox, prediction)

In [24]:
# (location, prediction)=face_detect_and_aligned('DSC_2201~2.JPG')

In [9]:
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Mask_detect_20201226_test.avi',fourcc, 30,(640,480))
while True:
    _, frame = cap.read()
    h,w = frame.shape[:2]
    frame =cv2.resize(frame,(640,480),fx=0.0,fy=0.0, interpolation = cv2.INTER_CUBIC)
    (location, prediction) = face_detect_and_aligned(frame)
        
    for (box, pred) in zip(location, prediction):
        (X,Y,X1,Y1) = box
        (mask, withoutMask) = pred
                    
        label = "Mask" if mask > 0.875 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            
#      To define the distance from camera, change the constant value 150 in the following if condition..
#think camera as person and I have used less distance you can increase more
        if X1-X>(150):
            # Check if prediction for without mask is 75% and above
            if withoutMask>0.8:
                cv2.putText(frame,'Alert !!! social Distancing!!!!',(X-50, Y - 50),cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,0,255),3)
        cv2.putText(frame, label, (X, Y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
        cv2.rectangle(frame, (X, Y), (X1, Y1), color, 2)
    cv2.imwrite('test_mask_detection_'+str(label)+'.png',frame)
#     out.write(frame)
    cv2.imshow("frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
#close poped up window
cv2.destroyAllWindows()
cap.release()
out.release()
